# Instalación de paquetes

In [2]:
%%capture
%pip install torch torchvision torchaudio easyocr pillow opencv-python ipywidgets

## Importar de librerias

In [51]:
from enum import Enum
import numpy as np
from collections import defaultdict
from matplotlib import pyplot as plt
from PIL import ImageGrab
import cv2
import easyocr
import ipywidgets as widgets
from IPython.display import display, clear_output

# Inicialización del Lector OCR

En esta celda, inicializamos un lector OCR (Reconocimiento Óptico de Caracteres) usando la biblioteca **EasyOCR**. Configuramos el lector para trabajar con el idioma español (`'es'`) y habilitamos el uso de la GPU (`gpu=True`) para acelerar el proceso de reconocimiento de texto en imágenes, siempre que haya una GPU disponible.

EasyOCR es una biblioteca que nos permite extraer texto de imágenes con alta precisión, y se ajusta automáticamente al idioma que especificamos. Aquí, seleccionamos español ya que es el idioma que nos interesa reconocer en las imágenes procesadas.

- **Idioma**: Español (`'es'`)
- **GPU**: Habilitado para un procesamiento más rápido en caso de que esté disponible.


In [3]:
reader=easyocr.Reader(['es'], gpu=True)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
C:\Users\Sanyval\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\easyocr\detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case 

# Solucion del problema

En este cuaderno, resolveremos el problema del **Buscaminas** usando una representación matricial del tablero. Para ello, necesitamos modelar dos clases principales: **`Celda`** y **`Tablero`**.

La clase `Celda` representa cada una de las posiciones del tablero, mientras que la clase `Tablero` es responsable de gestionar el conjunto de celdas, procesar la captura del tablero y aplicar el algoritmo para resolver el siguiente movimiento.

---

## Clase `Celda`

La clase `Celda` encapsula la información y el comportamiento de una única celda del tablero del Buscaminas. Esta clase almacena:
- La fila y columna en la que se encuentra la celda dentro del tablero.
- El valor de la celda, que puede indicar si es una bomba, está vacía, o tiene un número de bombas adyacentes.
- Los vecinos de la celda, que son otras celdas adyacentes las cuales estan cerradas.

### Atributos de la Clase `Celda`
- **`fila`**: La fila en la que se encuentra la celda en el tablero.
- **`columna`**: La columna en la que se encuentra la celda en el tablero.
- **`valor`**: El valor de la celda, que puede representar:
  - `-1`: Indica una celda descubierta (no tiene mina).
  - `0`: Indica una celda desconocida.
  - `1-8`: Indica el número de minas adyacentes.
- **`vecinos`**: Las celdas que están adyacentes (horizontal, vertical o diagonalmente) a la celda actual.
  
### Métodos de la Clase `Celda`
- **`__init__(self, fila, columna, valor, matriz)`**: Constructor que inicializa la celda con su posición, valor y obtiene sus vecinos.
- **`obtener_vecinos(self, matriz)`**: Método que devuelve una lista de las celdas vecinas (adyacentes).
- **`__repr__(self)`**: Método que devuelve una representación textual de la celda para su visualización.


In [4]:
class Celda:
    def __init__(self, fila, columna, valor,matriz):
        self.fila = fila
        self.columna = columna
        self.valor = valor
        self.vecinos = self.obtener_vecinos(matriz)

    def obtener_vecinos(self, matriz):
        vecinos = []
        for i in range(max(0, self.fila - 1), min(self.fila + 2, matriz.shape[0])):
            for j in range(max(0, self.columna - 1), min(self.columna + 2, matriz.shape[1])):
                if matriz[i][j]!=0:
                    continue

                if (i, j) != (self.fila, self.columna):
                    vecinos.append((i, j))
        return vecinos

    def __repr__(self):
        return f"Celda({self.fila}, {self.columna}, {self.valor},{len(self.vecinos)})"

## Clase `Tablero`: Métodos Principales

La clase `Tablero` es responsable de representar y gestionar el tablero del Buscaminas. Implementa una serie de métodos para capturar el tablero desde la pantalla, detectar las casillas, aplicar OCR para leer el tablero y finalmente calcular el siguiente movimiento utilizando una matriz de probabilidades.

### Atributos de la Clase `Tablero`
- **`filas`**: El número de filas en el tablero.
- **`columnas`**: El número de columnas en el tablero.

### Métodos Principales de la Clase `Tablero`

- **`__init__(self, filas, columnas)`**: Este método es el **constructor** de la clase. Inicializa el tablero con el número de filas y columnas especificado por el usuario.

- **`capturar_tablero(self, draw=False)`**: Este método **captura la pantalla** para obtener el tablero del Buscaminas. Realiza los siguientes pasos:
    - Captura la mitad izquierda de la pantalla.
    - Convierte la captura a escala de grises.
    - Detecta los contornos para identificar el área del tablero.
    - Recorta la imagen del tablero y, opcionalmente, la muestra usando `matplotlib` si `draw=True`.

Este método es clave para preparar la imagen del tablero para su procesamiento.

- **`_detectar_casillas_estado(self, tablero, estado=casilla_estado.ABIERTA, draw=False)`**:Este método detecta las **casillas abiertas o cerradas** del tablero basándose en su estado (abiertas o cerradas). Aplica técnicas de procesamiento de imágenes para encontrar los bordes de las casillas y devuelve las coordenadas de cada una.

- **`detectar_casillas(self, tolerancia_y=10, draw=False)`**:Este método **detecta todas las casillas del tablero** (abiertas y cerradas), las organiza en filas y columnas, y construye una **matriz de celdas** y una **matriz de estados** (abiertas o cerradas). También puede dibujar las casillas detectadas en la imagen si `draw=True`.

- **`leer_tablero(self, matriz_celdas=None)`**:Este método aplica **OCR (Reconocimiento Óptico de Caracteres)** para leer el valor de cada casilla del tablero, utilizando la imagen recortada de cada celda. Devuelve una matriz con los valores de cada celda (minas adyacentes o casillas vacías).

- **`siguiente_movimiento(self)`**:Este es el método clave que implementa la lógica para calcular el **siguiente movimiento** en el juego. Utiliza la matriz de celdas y una **matriz de probabilidades** para determinar qué casilla tiene la mayor probabilidad de ser segura. La lógica principal incluye:
    - Crear una lista de celdas con valores conocidos (adyacentes a minas).
    - Calcular la probabilidad de que cada casilla sea una mina, basada en los valores y vecinos.
    - Identificar la casilla con la probabilidad más alta de ser segura, sugiriendo ese movimiento como el siguiente.


In [34]:
class casilla_estado(Enum):
    CERRADA = 0
    ABIERTA= 1

class Tablero:
    def __init__(self, filas, columnas):
        self.filas = filas
        self.columnas = columnas
        self.tablero = None
    
    def __str__(self) -> str:
        return f"Tablero({self.filas}*{self.columnas})"
    
    def capturar_tablero(self, draw=False):

        #Captura la mitad izquierda del la pantalla (En formato RGB)
        screen = ImageGrab.grab()  # Captura toda la pantalla
        width, height = screen.size
        left_half = screen.crop((0, 0, width // 2, height))  # Recorta la mitad izquierda
        left_half=np.array(left_half)# Convierte la imagen en un array

        #Detectar el tablero del buscaminas
        gray = cv2.cvtColor(left_half, cv2.COLOR_RGB2GRAY)
        blurred=cv2.GaussianBlur(gray, (5, 5), 0)

        #Detectar los bordes y contornos
        edges = cv2.Canny(blurred, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Filtrar los contornos para encontrar el tablero
        board_contour = None
        for contour in contours:
            # Aproximar el contorno a un polígono
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Si el contorno tiene 4 lados y es suficientemente grande, es un rectángulo
            if len(approx) == 4 and cv2.contourArea(approx) > 10000:
                board_contour = approx
                break

        # Recortar el tablero de la imagen original
        if board_contour is not None:
            x, y, w, h = cv2.boundingRect(board_contour)
            cropped_image = left_half[y:y+h, x:x+w]

            # Mostrar la imagen recortada
            if draw:
                plt.figure(figsize=(6, 6))
                plt.imshow(cropped_image)
                plt.title("Tablero recortado")
                plt.show()

            self.tablero=cropped_image

    def _detectar_casillas_estado(self, tablero, estado=casilla_estado.ABIERTA, draw=False):
        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(tablero, cv2.COLOR_RGB2GRAY)

        # Aplicar un umbral adaptativo para binarizar la imagen si las casillas están abiertas
        if estado==casilla_estado.ABIERTA:
            adaptive_thresh= cv2.adaptiveThreshold(gray,255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,5,3)
            # Detectar los bordes de las casillas
            edges = cv2.Canny(adaptive_thresh, 175, 200)
            # Aplicar dilatación para cerrar pequeños huecos en los bordes
            kernel = np.ones((3, 3), np.uint8)
            edges_dilated = cv2.dilate(edges, kernel, iterations=1)
        else:
            # Aplicar detección de bordes
            edges = cv2.Canny(gray, 200, 200)
            # Aplicar dilatación para cerrar pequeños huecos en los bordes
            kernel = np.ones((2, 2), np.uint8)
            edges_dilated = cv2.dilate(edges, kernel, iterations=1)
        
        if draw:
            # Mostrar los bordes dilatados
            plt.figure(figsize=(6, 6))
            plt.imshow(edges_dilated, cmap='gray')
            plt.title("Bordes dilatados")
            plt.axis('off')
            plt.show()

        # Encontrar los contornos de las casillas
        contours, _ = cv2.findContours(edges_dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # Crear copia de la imagen para dibujar contornos si es necesario
        tablero_contornos = tablero.copy()
        celdas=[]

        for contour in contours:
            # Aproximar el contorno a un polígono
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Si el polígono tiene 4 lados, es un cuadrado o rectángulo
            if len(approx) == 4:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = w / float(h)
                area = w * h

                # Considerar solo contornos con una relación de aspecto cercana a 1 (cuadrado)
                if 0.9 <= aspect_ratio <= 1.1 and (300 if estado == casilla_estado.CERRADA else 100) <= area <= 5000:
                    # Verificar si la celda detectada se superpone con otra celda
                    overlap = False

                    if estado==casilla_estado.CERRADA:
                        for cx, cy, cw, ch in celdas:
                            if (abs(cx - x) < 10 and abs(cy - y) < 10) or (x >= cx and y >= cy and x+w <= cx+cw and y+h <= cy+ch):
                                overlap = True
                                break

                    if not overlap:
                        celdas.append((x, y, w, h))
                        # Dibujar un rectángulo alrededor de la celda detectada
                        cv2.rectangle(tablero_contornos, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        if draw:
            # Mostrar la imagen con las celdas delimitadas
            plt.figure(figsize=(10, 10))
            plt.imshow(tablero_contornos)
            plt.title("Celdas detectadas")
            plt.axis('off')
            plt.show()
        
        # Crear una lista de casillas con sus coordenadas y estado
        casillas=[(celda,estado)for celda in celdas]
        return casillas

    def detectar_casillas(self, tolerancia_y=10,draw=False):
        # Detectar las casillas abiertas y cerradas
        casillas_abiertas=self._detectar_casillas_estado(self.tablero, draw=draw)
        casillas_cerradas=self._detectar_casillas_estado(self.tablero, estado=casilla_estado.CERRADA, draw=draw)

        casillas= casillas_abiertas + casillas_cerradas
        filas_dict = defaultdict(list)

        # Agrupar celdas en filas según la coordenada y con una tolerancia
        for casilla,estado in casillas:
            y_coord = casilla[1]

            # Encuentra la fila adecuada
            for key in list(filas_dict.keys()):
                if abs(y_coord-key) <= tolerancia_y:
                    filas_dict[key].append((casilla,estado))
                    break
            else:
                # Si no se encuentra una fila adecuada, crea una nueva fila
                filas_dict[y_coord].append((casilla,estado))
        
        # Ordenar las celdas dentro de cada fila por la coordenada x
        for key in filas_dict:
            filas_dict[key] = sorted(filas_dict[key], key=lambda celda: celda[0][0])

        # Construir la matriz de celdas
        matriz = np.zeros((self.filas, self.columnas), dtype=object)
        # Construir la matriz de estados
        estados = np.zeros((self.filas, self.columnas), dtype=object)

        i = 0
        for key in sorted(filas_dict.keys()):  # Ordenar las filas por y antes de colocarlas en la matriz
            for celda in filas_dict[key]:
                fila = i // self.columnas
                columna = i % self.columnas
                matriz[fila, columna] = celda[0]
                estados[fila, columna] = celda[1]
                i += 1

        self.matriz_tablero=matriz
        self.estados_tablero=estados

        return matriz, estados
    
    def _leer_casilla(self,x,y,w,h):# OCR a una casilla
        roi= self.tablero[y:y+h, x:x+w]
        roi=cv2.cvtColor(roi, cv2.COLOR_RGB2BGR)
        
        text=reader.readtext(roi)
        return text

    def leer_tablero(self, matriz_celdas=None):# OCR al tablero completo del buscaminas
        if matriz_celdas is None:
            matriz_celdas= self.matriz_tablero
        matriz_valores= np.zeros((self.filas,self.columnas), dtype=int)

        for fila in range(self.filas):
            for columna in range(self.columnas):
                celda=matriz_celdas[fila,columna]
                x,y,w,h=celda
                # Aplicar OCR a la region de la casilla
                text=self._leer_casilla(x,y,w,h)
                if len(text)==0:# Si no se detecta texto, el valor es 0
                    value='0'
                else:
                    value=text[0][1]
                
                if value.isdigit():# Si el valor es un número, convertirlo a entero
                    value=int(value)
                
                if value==0:
                    if self.estados_tablero[fila,columna]==casilla_estado.ABIERTA:
                        value=-1
                
                # Almacenar el valor en matriz
                matriz_valores[fila,columna]=value
        
        self.valores_tablero=matriz_valores
        return matriz_valores
    
    def _encontrar_valores(self):
        celdas_valores=[]

        for fila in range(self.filas):
            for columna in range(self.columnas):
                if self.valores_tablero[fila,columna]>0:
                    celdas_valores.append(Celda(fila, columna,self.valores_tablero[fila,columna],self.valores_tablero))
        
        return celdas_valores
    
    def siguiente_movimiento(self):
        celda_valores=self._encontrar_valores() # Encontrar las celdas con valores
        # Inicializamos la matriz de probabilidades
        probabilidad=np.ones(self.valores_tablero.shape)
        
        for fila in range(self.filas):
            for columna in range(self.columnas):
                # Si la celda esta abierta (-1) o no es 0, la marcamos con un valor negativo para descartarlos de los siguientes movimientos
                if self.valores_tablero[fila, columna] != 0:
                    probabilidad[fila, columna] = -1
                else:# Si la celda no es vecina de alguna celda con valor, también la marcamos como negativa
                    es_vecina=False
                    for celda in celda_valores:
                        if(fila, columna) in celda.vecinos:
                            es_vecina=True
                            break
                    if not es_vecina:
                        probabilidad[fila,columna]=-1
        
        # Calcular la probabilidad de cada celda        
        for i in range(len(celda_valores)):
            celda_valores=sorted(celda_valores, key=lambda celda: (celda.valor, len(celda.vecinos)))# Ordenar las celdas con valores
            celda=celda_valores.pop(0)

            if celda.valor==0:
                for vecino in celda.vecinos:
                    for celda_v in celda_valores:
                        if vecino in celda_v.vecinos:
                            celda_v.vecinos.remove(vecino)
            elif celda.valor==len(celda.vecinos):
                for vecino in celda.vecinos:
                    fila,columna=vecino
                    if probabilidad[fila,columna]!=-1:
                        probabilidad[fila,columna]=0
                    
                    for celda_v in celda_valores:
                        if vecino in celda_v.vecinos:
                            celda_v.valor-=1
                            celda_v.vecinos.remove(vecino)
            else:
                probabilidad_mina=celda.valor/len(celda.vecinos)
                for vecino in celda.vecinos:
                    fila,columna=vecino
                    if probabilidad[fila,columna]!=-1:
                        probabilidad[fila,columna]=max(0,probabilidad[fila,columna]-probabilidad_mina)
            
        # Encontrar la celda con la probabilidad más alta
        max_probabilidad=0
        mejor_celda=None
        for fila in range(self.filas):
            for columna in range(self.columnas):
                if probabilidad[fila,columna]>max_probabilidad:
                    max_probabilidad=probabilidad[fila,columna]
                    mejor_celda=(fila,columna)

        # Si deseas visualizar la matriz de probabilidades y la celda con la probabilidad más alta, descomenta el siguiente código
        # # Crear una figura y un conjunto de ejes
        # fig, ax = plt.subplots()

        # # Crear una matriz de colores basada en los estados y probabilidades
        # colores = np.zeros(self.estados_tablero.shape + (3,))

        # for i in range(self.estados_tablero.shape[0]):
        #     for j in range(self.estados_tablero.shape[1]):
        #         if self.estados_tablero[i, j] == 0:  # Celda cerrada
        #             colores[i, j] = [0, 0, 1]  # Azul
        #             if probabilidad[i, j] == 0:  # Mina identificada
        #                 colores[i, j] = [1, 0, 0]  # Rojo
        #         else:  # Celda abierta
        #             colores[i, j] = [0.5, 0.5, 0.5]  # Gris

        #         # Colocar la probabilidad encima de cada celda
        #         prob_texto = f"{probabilidad[i, j]:.2f}"
        #         ax.text(j, i, prob_texto, ha='center', va='center', color='black', fontsize=8)
        # ax.imshow(colores, aspect='equal')
        # ax.axis('off')
        # plt.show()
        
        return mejor_celda
                        

# Interfaz Interactiva para Solucionar el Buscaminas

En esta sección, utilizamos `ipywidgets` para permitir al usuario especificar el número de filas y columnas del tablero del Buscaminas y generar un objeto `Tablero` interactivo basado en esos valores. Los widgets proporcionan una forma dinámica de interactuar con el cuaderno, lo que facilita la creación del tablero directamente desde la interfaz del usuario.

## Descripción de los Widgets Utilizados

### 1. **`input_filas`** y **`input_columnas`**
Estos son campos de texto interactivos de tipo `IntText` que permiten al usuario ingresar el número de filas y columnas para el tablero. Están preconfigurados con un valor inicial de `9`, que es el tamaño común de un tablero estándar del Buscaminas.

- **`input_filas`**: Campo para ingresar el número de filas del tablero.
- **`input_columnas`**: Campo para ingresar el número de columnas del tablero.

### 2. **Botón para Crear el Tablero**

El botón **"Crear Tablero"** ejecuta una función que crea una instancia de la clase `Tablero` con el número de filas y columnas ingresado por el usuario.

- **Descripción**: Muestra "Crear Tablero".
- **Estilo**: El botón tiene un estilo verde (`button_style='success'`), que indica que es una acción confirmatoria.
- **Función asociada**: Al hacer clic en el botón, se ejecuta la función `crear_tablero()`.

---

## Lógica del Código

### 1. **Función `crear_tablero(b)`**

Esta función toma el número de filas y columnas ingresado por el usuario y crea un objeto de la clase `Tablero` utilizando esos valores. La función utiliza el modificador `global` para asegurarse de que el objeto `tablero` sea accesible globalmente dentro del cuaderno de Jupyter.

- **`filas`**: Se obtiene el valor ingresado por el usuario en `input_filas`.
- **`columnas`**: Se obtiene el valor ingresado por el usuario en `input_columnas`.
- **`tablero`**: Se crea una instancia del objeto `Tablero` utilizando los valores de filas y columnas.

Después de crear el tablero, se imprime un mensaje indicando que el tablero ha sido creado, mostrando sus dimensiones.

### 2. **Asociar Evento `on_click`**

El método `on_click` vincula el botón "Crear Tablero" con la función `crear_tablero()`. De esta manera, cada vez que el usuario haga clic en el botón, se generará un nuevo objeto `Tablero`.

---

## Visualización de los Widgets

Finalmente, mostramos los campos de entrada y el botón utilizando la función `display()`, que permite que los widgets sean visibles e interactivos en el cuaderno de Jupyter.


In [56]:
# Definir un objeto Tablero
tablero=None

# Crear campos de entrada para filas y columnas
input_filas = widgets.IntText(
    value=9,  # Valor inicial
    description='Filas:',
    disabled=False
)

input_columnas = widgets.IntText(
    value=9,  # Valor inicial
    description='Columnas:',
    disabled=False
)

# Crear un botón para generar el tablero
boton = widgets.Button(
    description='Crear Tablero',
    disabled=False,
    button_style='success', 
    tooltip='Crear el Tablero',
    icon='check'
)

# Función que se ejecuta al hacer clic en el botón
def crear_tablero(b):
    global tablero
    filas = input_filas.value
    columnas = input_columnas.value
    tablero = Tablero(filas, columnas)
    print(f"Se ha creado: {tablero}")

# Asociar la función al evento de clic del botón
boton.on_click(crear_tablero)

# Mostrar los widgets
display(input_filas, input_columnas, boton)

IntText(value=9, description='Filas:')

IntText(value=9, description='Columnas:')

Button(button_style='success', description='Crear Tablero', icon='check', style=ButtonStyle(), tooltip='Crear …

Se ha creado: Tablero(9*9)


## Resolver el Tablero: Interfaz Interactiva

En esta sección, utilizamos widgets de **ipywidgets** para crear una interfaz interactiva que permita resolver el tablero del Buscaminas y mostrar el siguiente movimiento. La interfaz consta de dos botones principales: "Resolver" y "Finalizar", y una sección de salida donde se muestran los resultados.

---

### Descripción de los Elementos Interactivos

#### 1. **Label (`text`)**
Este widget de tipo `Label` muestra un mensaje al usuario: **"¿Desea resolver el tablero?"**, proporcionando contexto sobre las acciones disponibles en la interfaz.

#### 2. **Botones (`button_solve` y `button_finish`)**

##### a) `button_solve`:
Este botón tiene la etiqueta **"Resolver"** y está estilizado con un color verde (`button_style='success'`) para indicar una acción positiva o de confirmación. Al hacer clic, comienza el proceso para resolver el tablero.

##### b) `button_finish`:
Este botón tiene la etiqueta **"Finalizar"** y está estilizado con un color rojo (`button_style='danger'`). Su función es simplemente limpiar la salida y detener el proceso, mostrando un mensaje de **"Finalizado"**.

Ambos botones están organizados horizontalmente usando el contenedor `HBox`.

#### 3. **Output (`output`)**
El widget `output` se utiliza para mostrar la salida del proceso de resolución. Dependiendo de la acción del usuario, muestra:
- Un mensaje que indica que el cálculo está en progreso.
- El resultado con el siguiente movimiento y una imagen del tablero actualizada.

---

### Lógica del Código

#### 1. **Función `solve(b)`**

Esta función es ejecutada cuando el usuario hace clic en el botón **"Resolver"**. Dentro de ella:

1. **Mostrar mensaje inicial**: Antes de comenzar los cálculos, se limpia la salida previa y se muestra el mensaje **"Calculando Movimiento..."** para indicar que el proceso ha comenzado.
   
2. **Sub-función `calcular_movimiento()`**: 
   - Esta función encapsula el proceso principal de cálculo. Se ejecutan los métodos de la clase `Tablero` en el siguiente orden:
     - **`capturar_tablero()`**: Captura el tablero de la pantalla.
     - **`detectar_casillas()`**: Detecta las casillas abiertas y cerradas en el tablero.
     - **`leer_tablero()`**: Aplica OCR a las casillas para obtener su valor.
     - **`siguiente_movimiento()`**: Calcula el siguiente movimiento basado en la matriz de probabilidades.
   - Después del cálculo, el siguiente movimiento se resalta en una copia de la imagen del tablero, donde se dibuja un rectángulo en la casilla correspondiente. Finalmente, se muestra el tablero actualizado con una imagen y el movimiento sugerido.

3. **Salida**: El resultado del siguiente movimiento (su posición) y el tablero actualizado se muestran en el widget `output`.

---

#### 2. **Función `finish(b)`**

Esta función se ejecuta cuando se presiona el botón **"Finalizar"**. Simplemente limpia la salida y muestra el mensaje **"Finalizado"**, lo que indica que el proceso ha terminado.

---

### Flujo de la Interfaz

1. El usuario selecciona **"Resolver"**, lo que inicia el proceso de cálculo del siguiente movimiento.
2. Mientras el cálculo está en progreso, se muestra el mensaje **"Calculando Movimiento..."**.
3. Una vez que el cálculo termina, se muestra una imagen del tablero con el siguiente movimiento marcado en un recuadro rojo.
4. Si el usuario selecciona **"Finalizar"**, se limpia la salida y se muestra el mensaje **"Finalizado"**.

---

### Visualización de la Interfaz

Finalmente, se utilizan los widgets y se visualizan en la interfaz mediante `display()`.

- Se muestra el **texto** del label.
- Los **botones** están organizados horizontalmente mediante `HBox`.
- La **salida** dinámica se gestiona con el widget `output`.


In [57]:
text=widgets.Label(value="Desea resolver el tablero?")
button_solve = widgets.Button(description="Resolver", button_style='success')
button_finish = widgets.Button(description="Finalizar", button_style='danger')
botones = widgets.HBox([button_solve, button_finish])
output = widgets.Output()

def solve(b):
    with output:
        clear_output()
        print("Calculando Movimiento...")

    def calcular_movimiento():
        tablero.capturar_tablero()
        tablero.detectar_casillas()
        tablero.leer_tablero()
        resultado=tablero.siguiente_movimiento()
        with output:
            clear_output()
            # Crear imagen con el tablero y el siguiente movimiento
            tablero_img = tablero.tablero.copy()
            fila, columna = resultado
            x, y, w, h = tablero.matriz_tablero[fila, columna]
            cv2.rectangle(tablero_img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            plt.figure(figsize=(6, 6))
            plt.imshow(tablero_img)
            plt.title(f"Siguiente movimiento: ({fila+1},{columna+1})")
            plt.axis('off')
            plt.show()

    calcular_movimiento()

def finish(b):
    with output:
        clear_output()
        print("Finalizado")

button_solve.on_click(solve)
button_finish.on_click(finish)
display(text,botones,output)

Label(value='Desea resolver el tablero?')

Output()